## 코드
### 아래에 정리 후 삭제

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms
 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
print(device + " is available")
 
learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5
 
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor() 
    ])
)
test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)
 
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)
 
examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape
 
class ConvNet(nn.Module):
  def __init__(self): 
        super(ConvNet, self).__init__()

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # (주석 예) ? = 1, ? = 10, kernel size = 5, stribe = 1, ? = 0 인 ...
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) 
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False)
        self.mp = nn.MaxPool2d(2) 
        self.fc1 = nn.Linear(320,100) 
        self.fc2 = nn.Linear(100,10) 

  def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) 
        x = F.relu(self.mp(self.conv2(x))) 
        x = self.drop2D(x)
        x = x.view(x.size(0), -1) 
        x = self.fc1(x) 
        x = self.fc2(x) 
        return F.log_softmax(x) 
 
model = ConvNet().to(device) 
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
 
for epoch in range(epochs): 
    avg_cost = 0

    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() 
        # 모든 model의 gradient 값을 0으로 설정 - 초기화 이유 : ?
        hypothesis = model(data)
        cost = criterion(hypothesis, target) 
        cost.backward() 
        optimizer.step()
        avg_cost += cost / len(train_loader) 
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
 
model.eval()
with torch.no_grad(): 
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] 
        total += len(target) 
        correct += (preds==target).sum().item() 
        
    print('Test Accuracy: ', 100.*correct/total, '%')

# 1. 라이브러리 임포트 / 파라미터 설정


In [ ]:
import torch  #필요한 라이브러리 임포트
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms
 
device = 'cuda' if torch.cuda.is_available() else 'cpu' #device 정의
# 현재 환경에서 GPU 연산이 가능하다면 GPU 연산을 하고, 그렇지 않다면 CPU 연산을 하도록 한다.
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
print(device + " is available") # 어떤 연산이 가능한지 출력해준다.
 
learning_rate = 0.001 #하이퍼 파라미터를 변수로 둔다.
batch_size = 100
num_classes = 10
epochs = 5

cuda is available


# 2. 데이터셋 불러오기

In [ ]:
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST', #데이터를 다운로드 받을 경로
    train = True, #MNIST의 훈련 데이터를 리턴받음
    download = True, #해당 경로에 데이터가 없다면 다운로드 받겠다는 의미
    transform = transfroms.Compose([ 
        transfroms.ToTensor() #현재 데이터를 파이토치 텐서로 변환
    ])
)
test_set = torchvision.datasets.MNIST( 
    root = './data/MNIST',
    train = False, #테스트 데이터를 리턴받음
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw



# 3. 데이터로더 이용

In [ ]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
# 전체 데이터를 한번에 계산하는 것은 많은 계산량이 필요하므로
# 더 작은 단위로 나누어서 해당 단위로 학습하는 개념 >> 미니 배치 (배치 사이즈 존재 이유)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

# 4. 배치 인덱스, 데이터 생성

In [ ]:
examples = enumerate(train_set) #리스트의 원소에 순서값을 부여해주는 enumerate
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

torch.Size([1, 28, 28])

# 5. 모델 클래스 정의 / 옵티마이저 설정



In [ ]:
class ConvNet(nn.Module):
  def __init__(self): 
        super(ConvNet, self).__init__()

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # in_channels = 1, out_channels = 10, kernel size = 5, stribe = 1, padding = 0 인 Conv2d
        # Conv2d 는 input값에 2d covolution 연산을 적용하는 함수
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) 
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False)
        # 인접한 픽셀들은 입력 확률에 따라 모두 0 값을 가지거나 모두 유효한 값이 되도록
        self.mp = nn.MaxPool2d(2) #feature map의 크기를 줄여주고 중요 특징 강조
        self.fc1 = nn.Linear(320,100) # 저장된 가중치와 편향을 사용하여 입력에 선형 변환을 적용
        self.fc2 = nn.Linear(100,10) 

  def forward(self, x): #모델을 생성할 때, 신경망에 데이터를 지나가게 하는 함수 정의
        x = F.relu(self.mp(self.conv1(x))) # x를 relu 활성함수에 대입
        x = F.relu(self.mp(self.conv2(x))) 
        x = self.drop2D(x)
        x = x.view(x.size(0), -1) 
        x = self.fc1(x) 
        x = self.fc2(x) 
        return F.log_softmax(x) # x에 softmax를 적용 후 log 취한 것과 동일
        #softmax를 적용하면 elements들은 0 에서 1 사이의 확률을 갖게 됨.
 
model = ConvNet().to(device) 
criterion = nn.CrossEntropyLoss().to(device) #손실함수 > 획득한 결과와 실제 값 사이의 틀린 정도를 측정하여 이 값을 최소화하고자 함
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
# 옵티마이저 최적화 > 각 학습 단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정 

# 6. 학습

In [ ]:
for epoch in range(epochs): 
    avg_cost = 0

    for data, target in train_loader:
        data = data.to(device) #to()를 통해 텐서를 옮겨 사용
        target = target.to(device)
        optimizer.zero_grad() 
        # 모든 model의 gradient 값을 0으로 설정 
        #- 초기화 이유 : backward() 함수는 새로운 그래디언트를 기존 그래디언트에 누적시키므로
        hypothesis = model(data)
        cost = criterion(hypothesis, target) 
        cost.backward() # 손실 함수의 미분값, gradients 저장
        optimizer.step() # gradients 바탕으로 파라미터 조정
        avg_cost += cost / len(train_loader) 
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


[Epoch:    1] cost = 0.323142588
[Epoch:    2] cost = 0.104305863
[Epoch:    3] cost = 0.0808324292
[Epoch:    4] cost = 0.0694515482
[Epoch:    5] cost = 0.0603056625


# 7. 정확도 계산



In [ ]:
model.eval() # train time 과 test time 에서 수행하는 다른 작업을 수행할 수 있도록 스위치 하는 함수
#(evaluation 과정에서 사용하지 않아야 하는 layer들을 알아서 off)
with torch.no_grad(): 
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] 
        total += len(target) 
        correct += (preds==target).sum().item() 
        
    print('Test Accuracy: ', 100.*correct/total, '%')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Test Accuracy:  98.46 %
